In [ ]:
%load_ext autoreload
%autoreload 

import sys
sys.version
# sys.path.append('/usr/lib/python2.7')
# sys.path.append('/home/fizzer/catkin_ws/src/dvrk-ros/dvrk_python/src')
# sys.path.append('/home/fizzer/auto_cam_ws/devel/lib/python2.7/dist-packages')
# sys.path.append('/home/fizzer/catkin_ws/devel/lib/python2.7/dist-packages')
# sys.path

In [ ]:
import jupyros as jr
import rospy
import numpy as np
from sensor_msgs import msg
import cv2
import cv_bridge
from copy import deepcopy
import ipywidgets as widgets
import PIL.Image
import matplotlib.pyplot as plt
import dvrk
import PyKDL
import tf
import time
from tf_conversions import posemath
import utils

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
rospy.init_node('notebook')
rospy.get_published_topics()

In [ ]:
bridge = cv_bridge.CvBridge()
left_image = None
left_image_msg = None
left_camera_info = None

right_image = None
right_image_msg = None
right_camera_info = None

def left_image_callback(im_msg):
    global left_image, left_image_msg
    left_image = bridge.imgmsg_to_cv2(im_msg, desired_encoding='rgb8')
    left_image_msg = im_msg
    
def right_image_callback(im_msg):
    global right_image, right_image_msg
    right_image = bridge.imgmsg_to_cv2(im_msg, desired_encoding='rgb8')
    right_image_msg = im_msg
    
def left_camera_info_callback(camera_info_msg):
    global left_camera_info
    left_camera_info = camera_info_msg
    
def right_camera_info_callback(camera_info_msg):
    global right_camera_info
    right_camera_info = camera_info_msg
    
jr.subscribe('/stereo/left/image_raw', msg.Image, left_image_callback)
jr.subscribe('/stereo/left/camera_info', msg.CameraInfo, left_camera_info_callback)
jr.subscribe('/stereo/right/image_raw', msg.Image, right_image_callback)
jr.subscribe('/stereo/right/camera_info', msg.CameraInfo, right_camera_info_callback)

while left_image is None or right_image is None:
    if left_image is None:
        print("left image not receieved")
    if right_image is None:
        print("right image not receieved")
    time.sleep(0.5)

In [ ]:
plt.imshow(np.hstack((left_image, right_image)))

In [ ]:
tf_listener = tf.TransformListener()
time.sleep(1)
tf_listener.getFrameStrings()

In [ ]:
psm1 = dvrk.psm('PSM1')
psm2 = dvrk.psm('PSM2')
psm3 = dvrk.psm('PSM3')
ecm = dvrk.ecm('ECM')

psm1_suj = dvrk.suj('PSM2') #temp bypass to wrong suj matching betw psm1 and 2
psm2_suj = dvrk.suj('PSM1') #temp bypass to wrong suj matching betw psm1 and 2
psm3_suj = dvrk.suj('PSM3')
ecm_suj = dvrk.suj('ECM')

while ecm.get_current_position() == PyKDL.Frame() or ecm.get_desired_position() == PyKDL.Frame():
    time.sleep(0.5)
while psm1.get_current_position() == PyKDL.Frame() or psm1.get_desired_position() == PyKDL.Frame():
    time.sleep(0.5)

In [ ]:
from math import radians as r

#ECM_STARTING_JOINT_POS = np.asarray([-0.15669435,  0.17855662,  0.07069676,  0.17411496])
#ECM_STARTING_JOINT_POS = np.asarray([0.0615668 , 0.0523214 , 0.04854392, 0.15809197])

#one i chose - ECM_STARTING_JOINT_POS = np.asarray([-0.34478268, -0.1093848 ,  0.03261061,  0.143541  ])

#ECM_STARTING_JOINT_POS = np.asarray([-0.1, 0, 0,  0.03541  ])
#ECM_STARTING_JOINT_POS = np.asarray([0.8,0.3,0.48,-0.95])

#this value gave current ecm positions - (-22.918, -2.865, 80, -3.438) - need to ensure no need to offset ecm with direct control first

#ECM_STARTING_JOINT_POS = np.asarray([-0.4, -0.0 ,  0.05,  -0. ])

ECM_STARTING_JOINT_POS = np.asarray([0., -0.1 ,  0.05,  0.13541  ])
ecm.move_joint(ECM_STARTING_JOINT_POS)

# sm3 = [0.1, r(165), r(-60),r(-58), r(-11)]
# psm3_suj.move_joint(np.asarray(sm3))

In [ ]:
from math import radians as r

# sm1 = [0.5, -2.4958207607269287, 2.775073528289795, 0.13962633907794952, -0.5235987901687622]
# sm2 = [0.5, 2.6005406379699707, -2.7227137088775635, -0.20943951606750488, -0.7155849933624268]
# sm3 = [0.5, 2.3212878704071045, -1.378810167312622, -1.5707963705062866, -0.8901179432868958]
# cm = [1.0, -1.2042771577835083, 2.687807083129883, -1.4311699867248535]

# sm2 =  [0.5, 2.583087205886841, -2.7925267219543457, -0.10471975803375244, -0.6108652353286743]
# sm3 = [0.5, 2.2165682315826416, -1.4486232995986938, -1.5707963705062866, -1.0122909545898438]

sm1 = [0.5, -2.4958207607269287, 2.775073528289795, 0.13962633907794952, -0.5235987901687622]
# sm2 = [0.5, 2.6005406379699707, -2.7227137088775635, -0.20943951606750488, -0.7155849933624268]
# sm3 = [0.5, 2.3212878704071045, -1.378810167312622, -1.5707963705062866, -0.8901179432868958]
cm = [0.6, -1.2042771577835083, 2.687807083129883, -0.311699867248535]
#arm 2 and 3 having collisions still
# sm2 =  [0.5, 2.583087205886841, -2.7925267219543457, -0.10471975803375244, -0.6108652353286743]
# sm3 = [0.5, 2.2165682315826416, -1.4486232995986938, -1.5707963705062866, -0.90122909545898438]

#new parameters with no collision!!!
sm2 =  [0.5, r(111), r(-105), r(-55), r(-7)]
#sm3 = [0.09, r(93), r(82),r(-124), r(-2)]
sm3 = [r(0.1), r(145), r(-25), r(-80), r(-28), r(0)]

#psm1_suj.move_joint(np.asarray(sm2))
#psm2_suj.move_joint(np.asarray(sm1))
psm3_suj.move_joint(np.asarray(sm3))
#ecm_suj.move_joint(np.asarray(cm))

print(psm1_suj.get_current_joint_position())
print(psm2_suj.get_current_joint_position())
#print(psm3_suj.get_current_joint_position())
print(ecm_suj.get_current_joint_position())
print('----------------------------------')
print(psm1_suj.get_current_position())
print(psm2_suj.get_current_position())
#print(psm3_suj.get_current_position())
print(ecm_suj.get_current_position())

In [ ]:
def move_psm1_suj(a, b, c, d, e, f):
    psm1_suj.move_joint(np.asarray([a, r(b), r(c), r(d), r(e), r(f)]))
interact(move_psm1_suj, a=(0.0,1.0), b=(-180,180), c=(-180,180), d=(-180,180), e=(-180,180), f=(-180,180));

In [ ]:
def move_psm2_suj(a, b, c, d, e, f):
    psm2_suj.move_joint(np.asarray([a, r(b), c, r(d), r(e), r(f)]))
interact(move_psm2_suj, a=(0.0,1.0), b=(-180,180), c=(-180,180), d=(-180,180), e=(-180,180), f=(-180,180));

In [ ]:
from math import radians as r

def move_psm3_suj(a, b, c, d, e, f):
    psm3_suj.move_joint(np.asarray([a, r(b), r(c), r(d), r(e), r(f)]))
interact(move_psm3_suj, a=(0.0,1.0), b=(-180,180), c=(-180,180), d=(-180,180), e=(-180,180), f=(-180,180))


In [ ]:
from math import radians as r

def move_ecm_suj(a, b, c, d, e, f):
    ecm_suj.move_joint(np.asarray([a, b, c, d, e, f]))
interact(move_ecm_suj, a=(0.0,1.0), b=(0.0,1.0), c=(-1.0,1.0), d=(0.0,1.0), e=(0.0, 1.0), f=(0.0,1.0));

In [ ]:
time.sleep(1)
#array position A B C D E-claw orientation, F-claw orientation
# PSM_HOME_POS = np.asarray([0., 0.5, 0.5, 0.1, 0., 0.])

#third value to change height of the needle - cannula
PSM_HOME_POS = np.asarray([0., 0., 0.05, 0., 0., 0.])
psm1.move_joint(PSM_HOME_POS)
psm1.close_jaw()

psm2.move_joint(PSM_HOME_POS)
psm2.close_jaw()

#moving cannula to adjust PSM3 SUJ height offset
#PSM_HOME_POS3 = np.asarray([0., 0., 0.15, 0., 0., 0.])
psm3.move_joint(PSM_HOME_POS)
psm3.close_jaw()

In [ ]:
import image_geometry
utils = None
import utils
stereo_model = image_geometry.StereoCameraModel()
stereo_model.fromCameraInfo(left_camera_info, right_camera_info)

tf_cam_to_world = utils.tf_to_pykdl_frame(tf_listener.lookupTransform('simworld', 'simcamera', rospy.Time()))
tf_world_to_psm1 = \
    utils.tf_to_pykdl_frame(tf_listener.lookupTransform('PSM1_base', 'simworld', rospy.Time()))
tf_world_to_psm2 = \
   utils.tf_to_pykdl_frame(tf_listener.lookupTransform('PSM2_base', 'simworld', rospy.Time()))
tf_world_to_psm3 = \
   utils.tf_to_pykdl_frame(tf_listener.lookupTransform('PSM3_base', 'simworld', rospy.Time()))

objects, frame = utils.get_points_and_img(left_image_msg, right_image_msg, stereo_model, tf_cam_to_world)
plt.figure(figsize=(12, 5))
plt.imshow(frame)
for o in objects:
    o = PyKDL.Vector(o.x(), o.y(), o.z())
#     o = PyKDL.Vector(o.x(), o.y(), o.z())

In [ ]:
len(objects)

In [ ]:
# pair up points that are across from each other
# x is *more or less* the axis along the wound
paired_pts = []
while objects:
    pt = objects[0]
    objects.remove(pt)
#     pt2 = min(objects, key=lambda obj : abs(obj.x() - pt.x()))
    pt2 = min(objects, key=lambda obj : abs(obj.y() - pt.y()))
    objects.remove(pt2)
#     paired_pts.append(
#         (max(pt, pt2, key=lambda p: p.y()), min(pt, pt2, key=lambda p: p.y())))
    paired_pts.append(
        (max(pt, pt2, key=lambda p: p.x()), min(pt, pt2, key=lambda p: p.x())))
# print(paired_pts)
z_val = paired_pts[0][0][2]
print(len(paired_pts)*2-1)
for i in range(len(paired_pts)):
    paired_pts[i][0][2] = z_val
    paired_pts[i][1][2] = z_val
    
print(paired_pts[0][0][1])
print(paired_pts[1][0][1])
paired_pts = sorted(paired_pts,key=lambda p: float(p[0][1]))
print("paired points sorted:")
paired_pts

In [ ]:
import suturing_state_machine
# from importlib import reload
reload(suturing_state_machine)
reload(utils)
import time

pts_psm1 = paired_pts[:2]
pts_psm2 = paired_pts[4:]
pts_psm3 = paired_pts[2:4]

# pts_psm1 = paired_pts[3:6]
# pts_psm3 = paired_pts[2:4]
#pts_psm2 = paired_pts[3:6]

# double PSM same dir
# sm1 = suturing_state_machine.SuturingStateMachine(psm1, tf_world_to_psm1, pts_psm1, arm_name = 'PSM1')  
# sm2 = suturing_state_machine.SuturingStateMachine(psm2, tf_world_to_psm2, pts_psm2, arm_name = 'PSM2')

# start_time = time.clock()
# while not sm1.is_done() or not sm2.is_done():
#     sm1.run_once()
#     sm2.run_once()
#     time.sleep(0.1)
    
# completion_time = time.clock() - start_time
# print("completion time(s): ",completion_time)

# single PSM 
# pts_psm2 = paired_pts

# sm2 = suturing_state_machine.SuturingStateMachine(psm2, tf_world_to_psm2, pts_psm2, arm_name = 'PSM2')

# start_time = time.clock()
# while not sm2.is_done():
#     sm2.run_once()
#     time.sleep(0.1)
    
# completion_time = time.clock() - start_time
# print("completion time(s): ",completion_time)

# double PSM opposite dir
pts_psm1 = []
sm1 = suturing_state_machine.SuturingStateMachine(psm1, tf_world_to_psm1, pts_psm1, arm_name = 'PSM2')  
sm2 = suturing_state_machine.SuturingStateMachine(psm2, tf_world_to_psm2, pts_psm2, arm_name = 'PSM2')
sm3 = suturing_state_machine.SuturingStateMachine(psm3, tf_world_to_psm3, pts_psm3, arm_name = 'PSM2')

# for sublist in (paired_pts[:2]):
#     pts_psm1.append(sublist[::-1])
    
start_time = time.clock()
print("pts_psm1:", pts_psm1)
#print("pts_psm2:", pts_psm2)
# print("pts_psm3:", pts_psm3)

while not sm1.is_done() or not sm2.is_done() or not sm3.is_done():
#     if not sm1.is_done():
    sm1.run_once()
    sm2.run_once()
    sm3.run_once()
    #sm3.run_once()
#     if not sm2.is_done():
#         sm2.run_once()
#     if not sm3.is_done():
#         sm3.run_once()
    time.sleep(0.1)

completion_time = time.clock() - start_time
print("completion time(s): ",completion_time)

In [ ]:
dir(psm1)

In [ ]:
ecm.get_current_joint_position()

In [ ]:
psm2.close_jaw()